In [1]:
'''
Author: Gabriella Vidgen
Date: May 2023

Interactive slider which shows how the following results change with
varying the percentage of the two coordinated charging profiles:

- Generator Capacity
- Annual Energy Production
- Curtailment of Renewable Resources (Wind and Solar)
- Energy Production Over a Typical Day (5th April, 2050)
'''


import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def cost_calcs(folder):
    cost_file = f'{folder}costs.csv'
    cost_df = pd.read_csv(cost_file)
    cost_df = cost_df.iloc[1, 1] # Remove irrelevant columns and rows
    cost_df /= 10**9 # Change from $ to $ billion

    return cost_df

def emissions_calcs(folder):
    emissions_file = f'{folder}emissions.csv'
    emis_df = pd.read_csv(emissions_file)    
    emis_df = emis_df.iloc[2:, 2] # Remove irrelevant columns and rows
    total_emis = emis_df.sum() 
    total_emis /= 1000 # Mt to Gt

    return total_emis

def cap_calcs(folder):
    cap_file = f'{folder}capacity.csv'
    cap_df = pd.read_csv(cap_file)    
    cap_df = cap_df.iloc[:-1, [0, 2, 5]] # Remove irrelevant columns and rows    
    cap_df.iloc[:, [1, 2]] /= 1000 # MW to GW    
    cap_df.iloc[:, 0] = cap_df.iloc[:, 0].str.replace('QLD_', '') # Fix headings
    cap_labels = cap_df['Resource']
    start_cap = cap_df['StartCap']
    end_cap = cap_df['EndCap']

    return cap_labels, start_cap, end_cap

def curt_calcs(folder):
    power_file = f'{folder}power.csv'
    curt_file = f'{folder}curtail.csv'

    power_df = pd.read_csv(power_file)
    power_df.columns = power_df.columns.str.replace('[_QLD]', '', regex=True) # Fix headings

    total_used_wind = power_df.iloc[1, 6]
    total_used_solar = power_df.iloc[1, 7]

    curt_df = pd.read_csv(curt_file)
    curt_df.columns = curt_df.columns.str.replace('[_QLD]', '', regex=True) # Fix headings

    total_curt_wind = curt_df.iloc[1, 6]
    total_curt_solar = curt_df.iloc[1, 7]

    curt_wind_percent = 100 * (total_curt_wind)/(total_curt_wind + total_used_wind)
    curt_solar_percent = 100 * (total_curt_solar)/(total_curt_solar + total_used_solar)

    return curt_wind_percent, curt_solar_percent

def profit_calcs(folder):
    profit_file = f'{folder}NetRevenue.csv'
    prof_df = pd.read_csv(profit_file)    
    prof_df = prof_df.iloc[:, [1, -1]] # Remove irrelevant columns and rows    
    prof_df.iloc[:, -1] /= 10**6 # Change from $ to $ million    
    prof_df.iloc[:, 0] =prof_df.iloc[:, 0].str.replace('QLD_', '') # Fix headings

    prof_labels = prof_df.iloc[:, 0]
    prof_vals = prof_df.iloc[:, 1]

    return prof_labels, prof_vals

def energy_calcs(folder):
    power_file = f'{folder}power.csv'
    ener_df = pd.read_csv(power_file)
    ener_df = ener_df.iloc[[1], 1:-1]
    ener_df *= (8760/(1000)) # MW to GWh
    ener_df.columns = ener_df.columns.str.replace('[_QLD]', '', regex=True) # Fix headings

    ener_labels = ener_df.columns
    ener_vals = ener_df.iloc[0, :]

    return ener_labels, ener_vals

def six_fig(flex_percent):

    folder = f'Results2/{flex_percent}/'

    '''Process data'''

    # TYPICAL DAY - THURSDAY THE 31ST MARCH, 2050 (DAY 90/365)
    # start_ind = 95 * 24 + 2
    # end_ind = start_ind + 24
    # power_file = f'{folder}power.csv'
    # power_df = pd.read_csv(power_file)
    # power_df = power_df.iloc[start_ind: end_ind, 1:-1]
    # power_df /= 1000 # MW to GW
    # power_df.columns = power_df.columns.str.replace('[_QLD]', '', regex=True) # Fix headings
    # cumulative = power_df.cumsum(axis=1)
    # # define a list of colors
    # colors = ['yellow', 'limegreen', 'teal', 'pink', 'crimson', 'magenta', 'orange', 'darkviolet', 'lime', 'blue', 'cyan']

    day_index = 95 # 5th April
    start_ind = day_index * 24 + 2
    end_ind = start_ind + 24
    colors = ['yellow', 'limegreen', 'teal', 'pink', 'crimson', 'magenta', 'orange', 'darkviolet', 'lime', 'blue', 'cyan']

    folder1 = f'Results1/{flex_percent}/'
    power_file1 = f'{folder1}power.csv'
    power_df1 = pd.read_csv(power_file1)
    power_df1 = power_df1.iloc[start_ind: end_ind, 1:-1]
    power_df1 /= 1000 # MW to GW
    power_df1.columns = power_df1.columns.str.replace('[_QLD]', '', regex=True) # Fix headings
    cumulative1 = power_df1.cumsum(axis=1)

    folder2 = f'Results2/{flex_percent}/'
    power_file2 = f'{folder2}power.csv'
    power_df2 = pd.read_csv(power_file2)
    power_df2 = power_df2.iloc[start_ind: end_ind, 1:-1]
    power_df2 /= 1000 # MW to GW
    power_df2.columns = power_df2.columns.str.replace('[_QLD]', '', regex=True) # Fix headings
    cumulative2 = power_df2.cumsum(axis=1)



    '''Set up figure'''
    # Create the figure and axes
    fig, axs = plt.subplots(2, 3, figsize=(16, 7))

    cap_folder1 = f'Results1/{flex_percent}/'
    cap_folder2 = f'Results2/{flex_percent}/'
    cap_labels, start_cap, end_cap1 = cap_calcs(cap_folder1)
    _, _, end_cap2 = cap_calcs(cap_folder2)
    axs[1, 1].bar(np.arange(len(cap_labels))-0.3, start_cap, width=0.3, label='Start', color='orange')
    axs[1, 1].bar(np.arange(len(cap_labels)), end_cap1, width=0.3, label='End - Day \nPeak', color='blue')
    axs[1, 1].bar(np.arange(len(cap_labels))+0.3, end_cap2, width=0.3, label='End - Day \nand Night \nPeaks', color='magenta')
    # Adjust the figure size or the margins of the plot
    plt.subplots_adjust(bottom=0.25, right=0.95)
    # axs[1, 1].set_title('Total Annual Power Capacity')
    axs[1, 1].set_ylabel('Capacity (GW)')
    axs[1, 1].set_xticks(np.arange(len(cap_labels)), cap_labels, rotation=90)
    # plt.ylim(0, 25)
    axs[1, 1].legend(loc='upper left')


    curt_folder1 = f'Results1/{flex_percent}/'
    curt_folder2 = f'Results2/{flex_percent}/'
    curt_wind_percent1, curt_solar_percent1 = curt_calcs(curt_folder1)
    curt_wind_percent2, curt_solar_percent2 = curt_calcs(curt_folder2)
    curt_labels = ['wind', 'solar']
    axs[0, 0].bar(np.arange(len(curt_labels))-0.2, [curt_wind_percent1, curt_solar_percent1], width=0.35, label='Day Peak Profile', color='blue')
    axs[0, 0].bar(np.arange(len(curt_labels))+0.2, [curt_wind_percent2, curt_solar_percent2], width=0.35, label='Day and Night Peaks Profile', color='magenta')
    # Adjust the figure size or the margins of the plot
    # plt.subplots_adjust(bottom=0.25, right=0.95)
    axs[0, 0].set_title('Total Renewables Curtailment')
    axs[0, 0].set_ylim(0, 13)
    axs[0, 0].set_ylabel('Nameplate Capacity (%)')
    axs[0, 0].set_xticks(np.arange(len(curt_labels)), curt_labels)
    # axs[0, 0].legend(loc='upper left')


    prof_folder1 = f'Results1/{flex_percent}/'
    prof_folder2 = f'Results2/{flex_percent}/'
    prof_labels, prof_vals1= profit_calcs(prof_folder1)
    _, prof_vals2, = profit_calcs(prof_folder2)
    axs[1, 0].bar(np.arange(len(prof_labels))-0.2, prof_vals1, width=0.4, label='Day Peak', color='blue')
    axs[1, 0].bar(np.arange(len(prof_labels))+0.2, prof_vals2, width=0.4, label='Day and Night Peaks', color='magenta')
    axs[1, 0].set_xticks(np.arange(len(prof_labels)), prof_labels, rotation=90)
    axs[1, 0].set_ylabel('Annual Profit ($ million AUD)')
    axs[1, 0].set_ylim(0, 175)
    axs[1, 0].legend(loc='upper left')

    ener_folder1 = f'Results1/{flex_percent}/'
    ener_folder2 = f'Results2/{flex_percent}/'
    energy_labels, energy_vals1= energy_calcs(ener_folder1)
    _, energy_vals2, = energy_calcs(ener_folder2)
    axs[1, 2].bar(np.arange(len(energy_labels))-0.2, energy_vals1, width=0.4, label='Day Peak', color='blue')
    axs[1, 2].bar(np.arange(len(energy_labels))+0.2, energy_vals2, width=0.4, label='Day and \nNight Peaks', color='magenta')
    # Adjust the figure size or the margins of the plot
    plt.subplots_adjust(bottom=0.25, right=0.95)
    # axs[1, 2].set_title('Total Annual Energy Production')
    axs[1, 2].set_ylabel('Annual Energy Production (GWh)')
    axs[1, 2].set_xticks(np.arange(len(energy_labels)), energy_labels, rotation=90)
    axs[1, 2].set_ylim(0, 5 * 10**8)
    # axs[1, 2].legend(loc='upper left')


    # for i in range(power_df.shape[1]): 
    #     if i == 0:
    #         axs[0, 1].fill_between(power_df.index-start_ind, cumulative.iloc[:, i], 
    #                             alpha=0.5, label=power_df.columns[i], 
    #                             color=colors[i])
    #     else:
    #         axs[0, 1].fill_between(power_df.index-start_ind, cumulative.iloc[:, i], 
    #                             cumulative.iloc[:, i-1], alpha=0.5, 
    #                             label=power_df.columns[i], color=colors[i])


    # axs[0, 1].set_title('QLD Generation for 31/03/2050')
    # axs[0, 1].set_xlabel('Hour')
    # axs[0, 1].set_ylabel('Power Generated (GW)')
    # axs[0, 1].set_ylim(0, 22)
    # axs[0, 1].legend(loc='upper left', bbox_to_anchor=(1, 1))

   

    for i in range(power_df1.shape[1]): 
        if i == 0:
            axs[0, 1].fill_between(power_df1.index-start_ind, cumulative1.iloc[:, i], 
                                alpha=0.5, label=power_df1.columns[i], 
                                color=colors[i])
        else:
            axs[0, 1].fill_between(power_df1.index-start_ind, cumulative1.iloc[:, i], 
                                cumulative1.iloc[:, i-1], alpha=0.5, 
                                label=power_df1.columns[i], color=colors[i])

    axs[0, 1].set_title('Power Generation for a Typical Weekday \n(Day Peak Coordinated Charging Profile)', fontsize=10)
    axs[0, 1].set_xlabel('Hour')
    axs[0, 1].set_xticks(range(0, 24, 2))
    axs[0, 1].set_ylabel('Power Generated (GW)')
    axs[0, 1].set_ylim(0, 22)
    # axs[0, 1].legend(loc='upper left', bbox_to_anchor=(1.02, 1))
    # axs[0, 1].subplots_adjust(right=0.7)  # Adjust the right margin to accommodate the legend


    for i in range(power_df2.shape[1]): 
        if i == 0:
            axs[0, 2].fill_between(power_df2.index-start_ind, cumulative2.iloc[:, i], 
                                alpha=0.5, label=power_df2.columns[i], 
                                color=colors[i])
        else:
            axs[0, 2].fill_between(power_df2.index-start_ind, cumulative2.iloc[:, i], 
                                cumulative2.iloc[:, i-1], alpha=0.5, 
                                label=power_df2.columns[i], color=colors[i])

    axs[0, 2].set_title('Power Generation for a Typical Weekday \n(Day and Night Peaks Coordinated Charging Profile)', fontsize=10)
    axs[0, 2].set_xlabel('Hour')
    axs[0, 2].set_xticks(range(0, 24, 2))
    axs[0, 2].set_ylabel('Power Generated (GW)')
    axs[0, 2].set_ylim(0, 22)
    axs[0, 2].legend(loc='upper left', bbox_to_anchor=(1.02, 1))
    # axs[0, 2].subplots_adjust(right=0.7)  # Adjust the right margin to accommodate the legend

    # # Display the text displays
    # axs[0, 0].axis('off')
    # genx_obj_val = cost_calcs(folder)
    # emis_val = emissions_calcs(folder)
    # axs[0, 0].text(0.5, 0.5, f'GenX Objective \nFunction (AUD): \n ${genx_obj_val:.2f}b \n\n Total Emissions \n(CO2-e): \n{emis_val:.2f}Gt', ha='center', va='center', fontsize=20)

    # Set the overall title for the figure
    fig.suptitle(f'{flex_percent} Percent Coordinated Charging', fontsize=18)
    # plt.subplots_adjust(right=0.9) # make room for typical day legend
    plt.subplots_adjust(bottom=0.2, right=0.95)

    # Display the figure
    plt.show()

def update_fig(slider_value):
    for flex_percent in [30, 35, 40, 45, 50]:
        if slider_value == flex_percent:
            six_fig(flex_percent)

slider = widgets.IntSlider(min=30, max=50, step=5, value=30, description='Slider')
slider.layout = widgets.Layout(width='100%')
widgets.interact(update_fig, slider_value=slider)

interactive(children=(IntSlider(value=30, description='Slider', layout=Layout(width='100%'), max=50, min=30, s…

<function __main__.update_fig(slider_value)>